# DeepFM - no context

In [1]:
# Required modules

import pandas as pd
import tensorflow as tf

from itertools import product

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2023-04-26 23:19:40.869932: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-26 23:19:41.033556: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 23:19:41.780159: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 23:19:41.783129: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 23:19:45.291207: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Load the data

use_cols = ['user:token', 'item:token', 'cnt:float']
data = pd.read_csv('./dataset/frappe/frappe.inter', usecols=use_cols)
data.head()

,user:token,item:token,cnt:float
0,0,0,0
1,1,1,1
2,2,2,1
3,3,3,0
4,4,4,3


In [3]:
# Encoding the user_id column

user_encoder = LabelEncoder()
data['user:token'] = user_encoder.fit_transform(data['user:token'].values)

# Encoding the item_id column
item_encoder = LabelEncoder()
data['item:token'] = item_encoder.fit_transform(data['item:token'].values)

In [4]:
# Renaming columns

data.columns = ['user_id', 'item_id', 'rating']

In [5]:
# Split the data into train and test sets

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [6]:
# Define the number of unique users and movies

num_users = data['user_id'].nunique()
num_movies = data['item_id'].nunique()

# Define embedding size

embedding_size = 10

In [7]:
# Model definition

def get_model(embedding_size, weight_decay=0.0):
    # Define the input shape
    input_shape = (train_data.shape[1] - 1,)

    l2_reg = tf.keras.regularizers.l2(weight_decay)

    # Define input layers
    user_input = tf.keras.layers.Input(shape=(1,))
    movie_input = tf.keras.layers.Input(shape=(1,))

    # Define user embedding
    user_embedding = tf.keras.layers.Embedding(num_users, embedding_size, input_length=1)(user_input)
    user_embedding = tf.keras.layers.Flatten()(user_embedding)

    # Define movie embedding
    movie_embedding = tf.keras.layers.Embedding(num_movies, embedding_size, input_length=1)(movie_input)
    movie_embedding = tf.keras.layers.Flatten()(movie_embedding)

    # Concatenate user and movie embeddings
    concat = tf.keras.layers.concatenate([user_embedding, movie_embedding])

    # Define FM part
    fm = tf.keras.layers.Dense(1, activation=None)(concat)

    # Define DNN part
    dnn = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2_reg)(concat)
    dnn = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2_reg)(dnn)
    dnn = tf.keras.layers.Dense(1, activation=None)(dnn)

    # Concatenate FM and DNN parts
    concat = tf.keras.layers.concatenate([fm, dnn])

    # Define output layer
    output = tf.keras.layers.Flatten()(concat)

    # Define the model
    model = tf.keras.models.Model(inputs=[user_input, movie_input], outputs=output)

    return model

model = get_model(10)

In [8]:
# Compile the model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])

In [ ]:
# Fit the model

history = model.fit([train_data['user_id'], train_data['item_id']], 
                    train_data['rating'], 
                    validation_data=([test_data['user_id'], test_data['item_id']], test_data['rating']),
                    epochs=100, batch_size=64)

Epoch 1/100
1203/1203 [==============================] - 3s 2ms/step - loss: 1.1197 - mae: 0.8103 - val_loss: 0.8664 - val_mae: 0.7309
Epoch 2/100
1203/1203 [==============================] - 2s 2ms/step - loss: 0.7605 - mae: 0.6843 - val_loss: 0.6888 - val_mae: 0.6507
Epoch 3/100
1203/1203 [==============================] - 2s 2ms/step - loss: 0.6523 - mae: 0.6358 - val_loss: 0.6378 - val_mae: 0.6312
Epoch 4/100
1203/1203 [==============================] - 2s 2ms/step - loss: 0.6153 - mae: 0.6206 - val_loss: 0.6185 - val_mae: 0.6225
Epoch 5/100
1203/1203 [==============================] - 2s 2ms/step - loss: 0.5974 - mae: 0.6118 - val_loss: 0.6086 - val_mae: 0.6190
Epoch 6/100
1203/1203 [==============================] - 2s 2ms/step - loss: 0.5872 - mae: 0.6065 - val_loss: 0.6033 - val_mae: 0.6159
Epoch 7/100
 265/1203 [=====>........................] - ETA: 1s - loss: 0.5856 - mae: 0.6051

In [ ]:
# Evaluate the model

model.evaluate([test_data['user_id'], test_data['item_id']], test_data['rating'])

In [ ]:
# Plot the model

tf.keras.utils.plot_model(model, show_shapes=True)

## Cross validation

In [13]:
# Cross validation

from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

for train_index, valid_index in kf.split(train_data):
    train_set = train_data.iloc[train_index]
    valid_set = train_data.iloc[valid_index]
    
    model = get_model(embedding_size=10)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])
    
    model.fit([train_set['user_id'], train_set['item_id']], 
              train_set['rating'], 
              validation_data=([valid_set['user_id'], valid_set['item_id']], valid_set['rating']),
              epochs=100, batch_size=16, verbose=0)
    print(model.evaluate([test_data['user_id'], test_data['item_id']], test_data['rating']))

89/89 [==============================] - 0s 655us/step - loss: 2.0597 - mae: 1.0770
[2.0596837997436523, 1.0770288705825806]
89/89 [==============================] - 0s 697us/step - loss: 2.0669 - mae: 1.0940
[2.066863536834717, 1.0939524173736572]


KeyboardInterrupt: 

In [ ]:
# Hyperparameter tuning

from sklearn.model_selection import KFold

filename = 'frappe_hyper_deepfm_gs.csv'
hyper = pd.read_csv(f"./hypers/version2/{filename}")
learning_rate = [1e-6, 1e-5, 1e-3, 1e-1]
learner = ['adam', 'RMSprop']
epochs = [10, 20, 30]
embedding_size = [32, 128, 512]
weight_decay = [1e-3, 1e-1]
train_batch_size = [500, 1000]

for lr, learner, embedding_size, weight_decay, epoch, train_batch_size in product(learning_rate, learner, embedding_size, weight_decay, epochs, train_batch_size):
    kf = KFold(n_splits=5, random_state=42)

    for train_index, valid_index in kf.split(data):
        train_set = data.iloc[train_index]
        valid_set = data.iloc[valid_index]
        model = get_model(embedding_size=embedding_size)
        if learner == 'adam':
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse', metrics=['mae'])
        else:
            model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr), loss='mse', metrics=['mae'])
    
        model.fit([train_set['user_id'], train_set['item_id']], 
                  train_set['rating'], 
                  validation_data=([valid_set['user_id'], valid_set['item_id']], valid_set['rating']),
                  epochs=epoch, batch_size=train_batch_size, verbose=0)
        rmse, mae = model.evaluate([valid_set['user_id'], valid_set['item_id']], valid_set['rating'])
        hyper = pd.concat([hyper, pd.DataFrame(
            [[lr, learner, embedding_size, weight_decay, train_batch_size, mae, rmse]], columns=['learning_rate', 'learner', 'embedding_size', 'weight_decay', 'train_batch_size', 'epoch', 'mae', 'rmse']
        )])
        hyper.to_csv(f"./hypers/version2/{filename}", index=False)

602/602 [==============================] - 1s 1ms/step - loss: 0.5826 - mae: 0.5966
